This file is used for classification of emotion-based EEG signals from PD and HC using various Machine Learning (ML) algorithms. The input to ML algorithms are *Raw*, *CSP*, and *SPV* features. While *Raw* and *SPV* features are obtained using `matlab_files/eeg_csp_raw_run.m`, *SPV* features are obtained using `eeg_spectral.ipynb`. The detailed architecture, input and output dimensions are explained in the paper. The highlighted part in the following image indicate the pipeline of the current file.

Note: The configurations, like hyperparameter values, present in this file are not the final configurations as reported in the paper. The present configurations are maybe mutated for additional experiments. However, results reported in the paper can be reproduced by plugging in the appropriate configurations as mentioned in the paper.


![ML pipeline associated with the current notebook](images/ml.png)

We explore the following ML algorithms for the classification task:
1. k-Nearest Neighbour (kNN), where the test sample is assigned the label corresponding to the mode of its k-*closest* neighbours based on a suitable distance metric.
2. Support Vector Machine (SVM), where input data are transformed to a high-dimensional space where the two classes are linearly separable and the inter-class distance is maximum.
3. Gaussian Naive Bayes (GNB), a generative classifier assuming class-conditional feature independence.
4. Decision Tree (DT), which uses a tree-like graph structure where each leaf node represents a category label.
5. Linear Discriminant Analysis (LDA), which linearly transforms data to achieve maximal inter-class distance.
6. Logistic Regression (LR), which maps the input to class labels via the sigmoid function.

# Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.io as sio
import os
#from skopt import BayesSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
#from skopt.space import Real, Categorical, Integer
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Data

In [ ]:
# Path to pre-processed features
data = '../data/iir_feat_tensor_full_with_full_labels.mat'

# path to save final features, results, etc.
savePath = '../results/MLResults/IIR/'
#data = pd.read_csv(data, header=None)
#data.head()
#X = data.iloc[1:,0:-2]
#y = data.iloc[1:,-1]

# Load data
matContent = sio.loadmat(data)
X = matContent['nc_feat']
y = np.squeeze(matContent['nc_multi_labels'])

# Experiment name
experiment = 'MultiClass_IIR_NC'

filename = savePath+'ML_'+experiment+'.mat'

cm_title = 'Categorical Emotion Classification for IIR features - NC (NB)'

X = np.reshape(X, (X.shape[0],X.shape[1]*X.shape[2]))
del matContent

print(X.shape)

## Data Standardization and PCA

In [7]:
# Function for dimensionality reduction
def reduce_dim(X, y, pca_thresh, Scaler):
    y=y
    #y = np.ravel(y)
    if Scaler=="MinMax":
        from sklearn.preprocessing import MinMaxScaler
        sc = MinMaxScaler()
        X = sc.fit_transform(X)
    elif Scaler=="MaxAbs":
        from sklearn.preprocessing import MaxAbsScaler
        sc = MaxAbsScaler()
        X = sc.fit_transform(X)
    elif Scaler == "Standard":
        from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        X = sc.fit_transform(X)
    elif Scaler == "Normalizer":
        from sklearn.preprocessing import Normalizer
        sc = Normalizer()
        X = sc.fit_transform(X)
    else:
        pass

    pca = PCA(pca_thresh)
    X = pca.fit_transform(X)
    return X, y

# KNN Classifier

In [ ]:
# K Nearest Neighbors

# Parameter space for Grid search
depth = np.logspace(0.001,10)
param = {'KNC__n_neighbors':[2,3,4,5,6,7,8,9,10], 'KNC__weights':['uniform','distance'],
         'KNC__metric':['euclidean','chebychev','minkowski']}

# Initialise Stratified K Fold
kfold = StratifiedKFold(n_splits=10, shuffle=True)
 
cm_list = []
acc_list = [] 
wtd_f1_list = []
macro_f1_list = []
#conf_mat = np.zeros((2,2))
conf_mat = np.zeros((6,6))


for train_index, test_index in kfold.split(X, y): 
    X_train, X_test = X[train_index], X[test_index] 
    y_train, y_test = y[train_index], y[test_index]
    
    pipe = Pipeline([('scaler', StandardScaler()), ('pca', PCA(0.95)), ('KNC', KNeighborsClassifier())])
    estimator = GridSearchCV(estimator=pipe, param_grid=param, cv=kfold.split(X_train, y_train), verbose=2, n_jobs=-1)
    estimator.fit(X_train, y_train) 
    y_pred = estimator.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    acc_list.append(acc)
    wtd_f1 = f1_score(y_test, y_pred, average='weighted')
    macro_f1 = f1_score(y_test, y_pred, average='micro')
    wtd_f1_list.append(wtd_f1)
    macro_f1_list.append(macro_f1)
    #cm = confusion_matrix(y_test, y_pred, labels=[0,1])
    cm = confusion_matrix(y_test, y_pred, labels=[1,2,3,4,5,6])
    cm = cm/cm.sum(axis=1, keepdims=True)
    #conf_mat = conf_mat+cm
    cm_list.append(cm)

# Dump results
#conf_mat /= conf_mat.sum(axis=1, keepdims=True) #Normalised values in the CM
conf_mat = np.mean(cm_list, axis=0)
acc_mean = round(np.mean(acc_list),3)
acc_std = round(np.std(acc_list),3)
wtd_f1_mean = round(np.mean(wtd_f1_list),3)
wtd_f1_std = round(np.std(wtd_f1_list),3)
macro_f1_mean = round(np.mean(macro_f1_list),3)
macro_f1_std = round(np.std(macro_f1_list),3)
print('Accuracy:',acc_mean,'+-',acc_std)
print('Weighted FScore:',wtd_f1_mean,'+-',wtd_f1_std)
print('Macro FScore:',macro_f1_mean,'+-',macro_f1_std)

# Visualise results
#conf_mat = plot_confusion_matrix(estimator, X_test, y_test, cmap=plt.cm.Blues)   
#conf_mat.ax_.set_title('Conf_mat - Tree - HV vs LV - Full data')
fig, ax = plt.subplots(figsize=(10,8))
ax = sns.heatmap(conf_mat, cmap="YlGnBu", annot=True, fmt='.4f', vmin=0, 
                 vmax=1, annot_kws={"fontsize":12})
#ax.set_yticklabels(['LA','HA'],rotation = 0)
#ax.set_xticklabels(['LA','HA'],rotation = 0)
ax.set_yticklabels(['Sad', 'Happy', 'Fear', 'Disgust', 'Surprise', 'Anger'],rotation = 0)
ax.set_xticklabels(['Sad', 'Happy', 'Fear', 'Disgust', 'Surprise', 'Anger'],rotation = 0)
ax.set_title(cm_title)
ax.get_figure().savefig(filename[:-4]+'_conf_mat'+'.png')
plt.show()

# Save results
sio.savemat(filename, {'experiment':experiment, 'conf_mat':conf_mat, 'conf_mat_list': cm_list,
                       'best_params':best_params, 'acc_list':acc_list, 'wtd_f1_list':wtd_f1_list, 
                       'macro_f1_list':macro_f1_list})

# SVM Classifier

## Poly SVM

In [ ]:
#Poly SVM

param = {'SVC__C':[0.001,0.01,0.1,1,10,100,1000]}

# Initialise Stratified K Fold
kfold = StratifiedKFold(n_splits=10)

cm_list = []
acc_list = [] 
wtd_f1_list = []
macro_f1_list = []
conf_mat = np.zeros((2,2))
#conf_mat = np.zeros((6,6))

for train_index, test_index in kfold.split(X, y): 
    X_train, X_test = X[train_index], X[test_index] 
    y_train, y_test = y[train_index], y[test_index]
    pipe = Pipeline([('scaler', StandardScaler()), ('pca', PCA(0.95)), ('SVC', SVC())])
    estimator = GridSearchCV(estimator=pipe, param_grid=param, cv=kfold.split(X_train, y_train), verbose=2, n_jobs=-1)
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    acc_list.append(acc)
    wtd_f1 = f1_score(y_test, y_pred, average='weighted')
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    wtd_f1_list.append(wtd_f1)
    macro_f1_list.append(macro_f1)
    cm = confusion_matrix(y_test, y_pred, labels=[0,1])
    #cm = confusion_matrix(y_test, y_pred, labels=[1,2,3,4,5,6])
    #cm = cm/cm.sum(axis=1, keepdims=True)
    conf_mat = conf_mat+cm
    #cm_list.append(cm)

# Visualise results
#conf_mat /= conf_mat.sum(axis=1, keepdims=True) #Normalised values in the CM
conf_mat = np.mean(cm_list, axis=0)
acc_mean = round(np.mean(acc_list),3)
acc_std = round(np.std(acc_list),3)
wtd_f1_mean = round(np.mean(wtd_f1_list),3)
wtd_f1_std = round(np.std(wtd_f1_list),3)
macro_f1_mean = round(np.mean(macro_f1_list),3)
macro_f1_std = round(np.std(macro_f1_list),3)
print('Accuracy:',acc_mean,'+-',acc_std)
print('Weighted FScore:',wtd_f1_mean,'+-',wtd_f1_std)
print('Macro FScore:',macro_f1_mean,'+-',macro_f1_std)
#fig, ax = plt.subplots(figsize=(10,8))
#conf_mat = plot_confusion_matrix(estimator, X_test, y_test, cmap=plt.cm.Blues)   
#conf_mat.ax_.set_title('Conf_mat - Tree - HV vs LV - Full data')
ax = sns.heatmap(conf_mat, cmap="YlGnBu", annot=True, fmt='.4f', vmin=0, 
                 vmax=1, annot_kws={"fontsize":12})
ax.set_yticklabels(['NC','PD'],rotation = 0)
ax.set_xticklabels(['NC','PD'],rotation = 0)
#ax.set_yticklabels(['E1','E2','E3','E4','E5','E6'],rotation = 0)
#ax.set_xticklabels(['E1','E2','E3','E4','E5','E6'],rotation = 0)
ax.set_title(cm_title)
ax.get_figure().savefig(filename[:-4]+'_conf_mat'+'.png')
plt.show()

# Save results
sio.savemat(filename, {'conf_mat':conf_mat, 'conf_mat_list': cm_list, 'best_params':best_params,
                       'acc_list':acc_list, 'wtd_f1_list':wtd_f1_list, 'macro_f1_list':macro_f1_list})

## Linear SVM

In [ ]:
#Linear SVM

param = {'LSVC__C':[0.001,0.01,0.1,1,10,100,1000]}

# Initialise Stratified K Fold
kfold = StratifiedKFold(n_splits=10)

acc_list = [] 
wtd_f1_list = []
macro_f1_list = []
conf_mat = np.zeros((2,2))
#conf_mat = np.zeros((6,6))
for train_index, test_index in kfold.split(X, y): 
    X_train, X_test = X[train_index], X[test_index] 
    y_train, y_test = y[train_index], y[test_index]
    pipe = Pipeline([('scaler', StandardScaler()), ('pca', PCA(0.95)), ('LSVC', LinearSVC())])
    estimator = GridSearchCV(estimator=pipe, param_grid=param, cv=kfold.split(X_train, y_train), verbose=2, n_jobs=-1)
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    acc_list.append(acc)
    wtd_f1 = f1_score(y_test, y_pred, average='weighted')
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    wtd_f1_list.append(wtd_f1)
    macro_f1_list.append(macro_f1)
    cm = confusion_matrix(y_test, y_pred, labels=[1,2,3,4,5,6])
    conf_mat = conf_mat+cm
    #cm_list.append(cm)

#conf_mat /= conf_mat.sum(axis=1, keepdims=True) #Normalised values in the CM
conf_mat = np.mean(cm_list, axis=0)
acc_mean = round(np.mean(acc_list),4)
acc_std = round(np.std(acc_list),4)
wtd_f1_mean = round(np.mean(wtd_f1_list),4)
wtd_f1_std = round(np.std(wtd_f1_list),4)
macro_f1_mean = round(np.mean(macro_f1_list),4)
macro_f1_std = round(np.std(macro_f1_list),4)
print('Accuracy:',acc_mean,'+-',acc_std)
print('Weighted FScore:',wtd_f1_mean,'+-',wtd_f1_std)
print('Macro FScore:',macro_f1_mean,'+-',macro_f1_std)

# Visualise results

#conf_mat = plot_confusion_matrix(estimator, X_test, y_test, cmap=plt.cm.Blues)   
#conf_mat.ax_.set_title('Conf_mat - Tree - HV vs LV - Full data')
#fig, ax = plt.subplots(figsize=(10,8))
ax = sns.heatmap(conf_mat, cmap="YlGnBu", annot=True, fmt='.4f', vmin=0, 
                 vmax=1, annot_kws={"fontsize":12})
ax.set_yticklabels(['LA','HA'],rotation = 0)
ax.set_xticklabels(['LA','HA'],rotation = 0)
#ax.set_yticklabels(['E1','E2','E3','E4','E5','E6'],rotation = 0)
#ax.set_xticklabels(['E1','E2','E3','E4','E5','E6'],rotation = 0)
ax.set_title(cm_title)
ax.get_figure().savefig(filename)
plt.show()

# Save results
sio.savemat(filename, {'conf_mat':conf_mat, 'conf_mat_list': cm_list, 'best_params':best_params,
                       'acc_list':acc_list, 'wtd_f1_list':wtd_f1_list, 'macro_f1_list':macro_f1_list})

# Decision Tree classifier

In [ ]:
# Decision Tree Classifier

# Grid search space
depth = np.logspace(0.01,10)
param = {'DTC__criterion':['gini','entropy'], 'DTC__splitter':['best'], 'DTC__max_depth':depth,
         'DTC__max_features': ['auto', 'sqrt', 'log2']}

# Initialise Stratified K Fold
kfold = StratifiedKFold(n_splits=10)

acc_list = [] 
wtd_f1_list = []
macro_f1_list = []
conf_mat = np.zeros((2,2))
#conf_mat = np.zeros((6,6))
for train_index, test_index in kfold.split(X, y): 
    X_train, X_test = X[train_index], X[test_index] 
    y_train, y_test = y[train_index], y[test_index]
    pipe = Pipeline([('scaler', StandardScaler()), ('pca', PCA(0.95)), ('DTC', DecisionTreeClassifier())])
    estimator = GridSearchCV(estimator=pipe, param_grid=param, cv=kfold.split(X_train, y_train), verbose=2, n_jobs=-1)
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    acc_list.append(acc)
    wtd_f1 = f1_score(y_test, y_pred, average='weighted')
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    wtd_f1_list.append(wtd_f1)
    macro_f1_list.append(macro_f1)
    cm = confusion_matrix(y_test, y_pred, labels=[0,1])
    #cm = confusion_matrix(y_test, y_pred, labels=[1,2,3,4,5,6])
    conf_mat = conf_mat+cm

conf_mat /= conf_mat.sum(axis=1, keepdims=True)
acc_mean = round(np.mean(acc_list),4)
acc_std = round(np.std(acc_list),4)
wtd_f1_mean = round(np.mean(wtd_f1_list),4)
wtd_f1_std = round(np.std(wtd_f1_list),4)
macro_f1_mean = round(np.mean(macro_f1_list),4)
macro_f1_std = round(np.std(macro_f1_list),4)
print('Accuracy:',acc_mean,'+-',acc_std)
print('Weighted FScore:',wtd_f1_mean,'+-',wtd_f1_std)
print('Macro FScore:',macro_f1_mean,'+-',macro_f1_std)

# Visualise results

#conf_mat = plot_confusion_matrix(estimator, X_test, y_test, cmap=plt.cm.Blues)   
#conf_mat.ax_.set_title('Conf_mat - Tree - HV vs LV - Full data')
#fig, ax = plt.subplots(figsize=(10,8))
ax = sns.heatmap(conf_mat, cmap="YlGnBu", annot=True, fmt='.4f', vmin=0, 
                 vmax=1, annot_kws={"fontsize":12})
ax.set_yticklabels(['NC','PD'],rotation = 0)
ax.set_xticklabels(['NC','PD'],rotation = 0)
#ax.set_yticklabels(['E1','E2','E3','E4','E5','E6'],rotation = 0)
#ax.set_xticklabels(['E1','E2','E3','E4','E5','E6'],rotation = 0)
ax.set_title(cm_title)
ax.get_figure().savefig(filename)
plt.show()

# Save results
sio.savemat(filename, {'conf_mat':conf_mat, 'conf_mat_list': cm_list, 'best_params':best_params,
                       'acc_list':acc_list, 'wtd_f1_list':wtd_f1_list, 'macro_f1_list':macro_f1_list})

# Discriminant Classifier

## Linear Discriminant Analysis

In [ ]:
# Linear Discriminant Analysis:

param = {'LDA__solver':['svd','lsqr','eigen']}

kfold = StratifiedKFold(n_splits=10)

acc_list = [] 
wtd_f1_list = []
macro_f1_list = []
#conf_mat = np.zeros((2,2))
conf_mat = np.zeros((6,6))
for train_index, test_index in kfold.split(X, y): 
    X_train, X_test = X[train_index], X[test_index] 
    y_train, y_test = y[train_index], y[test_index]
    pipe = Pipeline([('scaler', StandardScaler()), ('pca', PCA(0.95)), ('LDA', LinearDiscriminantAnalysis())])
    estimator = GridSearchCV(estimator=pipe, param_grid=param, cv=kfold.split(X_train, y_train), verbose=2, n_jobs=-1)
    estimator.fit(X_train, y_train) 
    y_pred = estimator.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    acc_list.append(acc)
    wtd_f1 = f1_score(y_test, y_pred, average='weighted')
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    wtd_f1_list.append(wtd_f1)
    macro_f1_list.append(macro_f1)
    #cm = confusion_matrix(y_test, y_pred, labels=[0,1])
    cm = confusion_matrix(y_test, y_pred, labels=[1,2,3,4,5,6])
    conf_mat = conf_mat+cm

conf_mat /= conf_mat.sum(axis=1, keepdims=True)
acc_mean = round(np.mean(acc_list),4)
acc_std = round(np.std(acc_list),4)
wtd_f1_mean = round(np.mean(wtd_f1_list),4)
wtd_f1_std = round(np.std(wtd_f1_list),4)
macro_f1_mean = round(np.mean(macro_f1_list),4)
macro_f1_std = round(np.std(macro_f1_list),4)
print('Accuracy:',acc_mean,'+-',acc_std)
print('Weighted FScore:',wtd_f1_mean,'+-',wtd_f1_std)
print('Macro FScore:',macro_f1_mean,'+-',macro_f1_std)
#conf_mat = plot_confusion_matrix(estimator, X_test, y_test, cmap=plt.cm.Blues)   
#conf_mat.ax_.set_title('Conf_mat - Tree - HV vs LV - Full data')
fig, ax = plt.subplots(figsize=(10,8))
ax = sns.heatmap(conf_mat, cmap="YlGnBu", annot=True, fmt='.4f', vmin=0, 
                 vmax=1, annot_kws={"fontsize":12})
#ax.set_yticklabels(['NC','PD'],rotation = 0)
#ax.set_xticklabels(['NC','PD'],rotation = 0)
ax.set_yticklabels(['E1','E2','E3','E4','E5','E6'],rotation = 0)
ax.set_xticklabels(['E1','E2','E3','E4','E5','E6'],rotation = 0)
ax.set_title(cm_title)
ax.get_figure().savefig(filename)
plt.show()

# Save results
sio.savemat(filename, {'conf_mat':conf_mat, 'conf_mat_list': cm_list, 'best_params':best_params,
                       'acc_list':acc_list, 'wtd_f1_list':wtd_f1_list, 'macro_f1_list':macro_f1_list})

## Quadratic Discriminant

In [ ]:
#Quadratic Discriminant

param = {'QDA__reg_param':[0.001,0.01,0.1,1,10,100,1000]}


kfold = StratifiedKFold(n_splits=10)

acc_list = [] 
wtd_f1_list = []
macro_f1_list = []
conf_mat = np.zeros((2,2))
for train_index, test_index in kfold.split(X, y): 
    X_train, X_test = X[train_index], X[test_index] 
    y_train, y_test = y[train_index], y[test_index]
    pipe = Pipeline([('scaler', StandardScaler()), ('pca', PCA(0.95)), ('QDA', QuadraticDiscriminantAnalysis())])
    estimator = GridSearchCV(estimator=pipe, param_grid=param, cv=kfold.split(X_train, y_train), verbose=2, n_jobs=-1)
    estimator.fit(X_train, y_train)  
    y_pred = estimator.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    acc_list.append(acc)
    wtd_f1 = f1_score(y_test, y_pred, average='weighted')
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    wtd_f1_list.append(wtd_f1)
    macro_f1_list.append(macro_f1)
    cm = confusion_matrix(y_test, y_pred, labels=[0,1])
    conf_mat = conf_mat+cm

conf_mat /= conf_mat.sum(axis=1, keepdims=True)
acc_mean = round(np.mean(acc_list),4)
acc_std = round(np.std(acc_list),4)
wtd_f1_mean = round(np.mean(wtd_f1_list),4)
wtd_f1_std = round(np.std(wtd_f1_list),4)
macro_f1_mean = round(np.mean(macro_f1_list),4)
macro_f1_std = round(np.std(macro_f1_list),4)
print('Accuracy:',acc_mean,'+-',acc_std)
print('Weighted FScore:',wtd_f1_mean,'+-',wtd_f1_std)
print('Macro FScore:',macro_f1_mean,'+-',macro_f1_std)
#conf_mat = plot_confusion_matrix(estimator, X_test, y_test, cmap=plt.cm.Blues)   
#conf_mat.ax_.set_title('Conf_mat - Tree - HV vs LV - Full data')
#fig, ax = plt.subplots(figsize=(10,8))
ax = sns.heatmap(conf_mat, cmap="YlGnBu", annot=True, fmt='.4f', vmin=0, 
                 vmax=1, annot_kws={"fontsize":12})
ax.set_yticklabels(['NC','PD'],rotation = 0)
ax.set_xticklabels(['NC','PD'],rotation = 0)
ax.set_title(cm_title)
ax.get_figure().savefig(filename)
plt.show()

# Save results
sio.savemat(filename, {'conf_mat':conf_mat, 'conf_mat_list': cm_list, 'best_params':best_params,
                       'acc_list':acc_list, 'wtd_f1_list':wtd_f1_list, 'macro_f1_list':macro_f1_list})

# Naive Bayes CLassifier 

In [ ]:
# Naive Bayes
param = {'GNB__var_smoothing': [0.0001,0.001,0.01,0.1,1,10,100,100,1000]}

kfold = StratifiedKFold(n_splits=10)

cm_list = []
acc_list = [] 
wtd_f1_list = []
macro_f1_list = []
#conf_mat = np.zeros((2,2))
conf_mat = np.zeros((6,6))
for train_index, test_index in kfold.split(X, y): 
    X_train, X_test = X[train_index], X[test_index] 
    y_train, y_test = y[train_index], y[test_index]
    pipe = Pipeline([('scaler', StandardScaler()), ('pca', PCA(0.95)), ('GNB', GaussianNB())])
    estimator = GridSearchCV(estimator=pipe, param_grid=param, cv=kfold.split(X_train, y_train), verbose=2, n_jobs=-1)
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    acc_list.append(acc)
    wtd_f1 = f1_score(y_test, y_pred, average='weighted')
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    wtd_f1_list.append(wtd_f1)
    macro_f1_list.append(macro_f1)
    #cm = confusion_matrix(y_test, y_pred, labels=[0,1])
    cm = confusion_matrix(y_test, y_pred, labels=[1,2,3,4,5,6])
    cm = cm/cm.sum(axis=1, keepdims=True)
    #conf_mat = conf_mat+cm
    cm_list.append(cm)

#conf_mat /= conf_mat.sum(axis=1, keepdims=True) #Normalised values in the CM
conf_mat = np.mean(cm_list, axis=0)
acc_mean = round(np.mean(acc_list),3)
acc_std = round(np.std(acc_list),3)
wtd_f1_mean = round(np.mean(wtd_f1_list),3)
wtd_f1_std = round(np.std(wtd_f1_list),3)
macro_f1_mean = round(np.mean(macro_f1_list),3)
macro_f1_std = round(np.std(macro_f1_list),3)
print('Accuracy:',acc_mean,'+-',acc_std)
print('Weighted FScore:',wtd_f1_mean,'+-',wtd_f1_std)
print('Macro FScore:',macro_f1_mean,'+-',macro_f1_std)

# Visualise results
#conf_mat = plot_confusion_matrix(estimator, X_test, y_test, cmap=plt.cm.Blues)   
#conf_mat.ax_.set_title('Conf_mat - Tree - HV vs LV - Full data')
fig, ax = plt.subplots(figsize=(10,8))
ax = sns.heatmap(conf_mat, cmap="YlGnBu", annot=True, fmt='.4f', vmin=0, 
                 vmax=1, annot_kws={"fontsize":12})
#ax.set_yticklabels(['LA','HA'],rotation = 0)
#ax.set_xticklabels(['LA','HA'],rotation = 0)
ax.set_yticklabels(['Sad', 'Happy', 'Fear', 'Disgust', 'Surprise', 'Anger'],rotation = 0)
ax.set_xticklabels(['Sad', 'Happy', 'Fear', 'Disgust', 'Surprise', 'Anger'],rotation = 0)
ax.set_title(cm_title)
ax.get_figure().savefig(filename[:-4]+'_conf_mat'+'.png')
plt.show()

# Save results
sio.savemat(filename, {'conf_mat':conf_mat, 'conf_mat_list': cm_list, 'best_params':best_params,
                       'acc_list':acc_list, 'wtd_f1_list':wtd_f1_list, 'macro_f1_list':macro_f1_list})

# Logistic Regression

In [ ]:
#Logistic Regression

param = {'LR__penalty':['l2','none'],'LR__C':[0.001,0.01,0.1,1,10,100,1000]}

kfold = StratifiedKFold(n_splits=10)

acc_list = [] 
wtd_f1_list = []
macro_f1_list = []
conf_mat = np.zeros((2,2))
for train_index, test_index in kfold.split(X, y): 
    X_train, X_test = X[train_index], X[test_index] 
    y_train, y_test = y[train_index], y[test_index]
    pipe = Pipeline([('scaler', StandardScaler()), ('pca', PCA(0.95)), ('LR', LogisticRegression())])
    estimator = GridSearchCV(estimator=pipe, param_grid=param, cv=kfold.split(X_train, y_train), verbose=2, n_jobs=-1)
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    acc_list.append(acc)
    wtd_f1 = f1_score(y_test, y_pred, average='weighted')
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    wtd_f1_list.append(wtd_f1)
    macro_f1_list.append(macro_f1)
    cm = confusion_matrix(y_test, y_pred, labels=[0,1])
    conf_mat = conf_mat+cm

conf_mat /= conf_mat.sum(axis=1, keepdims=True)
acc_mean = round(np.mean(acc_list),4)
acc_std = round(np.std(acc_list),4)
wtd_f1_mean = round(np.mean(wtd_f1_list),4)
wtd_f1_std = round(np.std(wtd_f1_list),4)
macro_f1_mean = round(np.mean(macro_f1_list),4)
macro_f1_std = round(np.std(macro_f1_list),4)
print('Accuracy:',acc_mean,'+-',acc_std)
print('Weighted FScore:',wtd_f1_mean,'+-',wtd_f1_std)
print('Macro FScore:',macro_f1_mean,'+-',macro_f1_std)
#conf_mat = plot_confusion_matrix(estimator, X_test, y_test, cmap=plt.cm.Blues)   
#conf_mat.ax_.set_title('Conf_mat - Tree - HV vs LV - Full data')
#fig, ax = plt.subplots(figsize=(10,8))
ax = sns.heatmap(conf_mat, cmap="YlGnBu", annot=True, fmt='.4f', vmin=0, 
                 vmax=1, annot_kws={"fontsize":12})
ax.set_yticklabels(['NC','PD'],rotation = 0)
ax.set_xticklabels(['NC','PD'],rotation = 0)
ax.set_title(cm_title)
ax.get_figure().savefig(filename)
plt.show()